In [1]:
from notebook_log_loading import load_sensor_log_file

sensor_log,first_timestamp=load_sensor_log_file("../to_study/initial_motion_calibration_2.txt")

In [4]:
import sys, os
parent_path = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)))
#print(path2add)
sys.path.append(parent_path)
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii")))
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii","underpinnings")))
from collections import deque
from gyrii.Gyrus import ThreadedGyrus
from gyrii.underpinnings.NNCalibration2 import StatePredictorWithPolicy
import numpy as np

def toeuler(q):
    w=q[3]
    i=q[0]
    j=q[1]
    k=q[2]
    s=1
    
    cpch=1-2*s*(j*j+k*k)
    cpsh=2*s*(i*j+k*w)
    sp=2*s*(i*k-j*w)
    sbcp=2*s*(j*k+i*w)    
    cbcp=1-2*s*(i*i+j*j)
    
    conv=360/(2*np.pi)
    #heading=360*np.arctan2(cpsh,cpch)/(2*3.14)
    #pitch=-360*np.arcsin(sp)/(2*3.14)
    #bank=360*np.arctan2(sbcp,cbcp)/(2*3.15)
    heading=conv*np.arctan2( 2*(w*i+j*k),1-2*(i*i+j*j))
    pitch=conv*np.arcsin(np.clip(2*(w*j-k*i),-1,1))    
    bank=conv*np.arctan2( 2*(w*k+i*j),1-2*(j*j+k*k))
    return [heading,pitch,bank]

def get_unique_tagged_objects(tagged_objects,objects_to_watch):
        ret={}
        label_list=[ x["label"] for x in tagged_objects]
        for key in objects_to_watch:
            if label_list.count(key)>1 or label_list.count(key)==0:
                continue
            elem=next(filter(lambda x: x["label"]==key,tagged_objects))
            ret[key]=elem
        return ret

class VisualMotionCalibGyrus(ThreadedGyrus):
    def __init__(self,broker,load_file=None):

        #self.motionpredictor=nn.Sequential(nn.Linear(4,4),
        #                      nn.Tanh(),
        #                      nn.Linear(4,2))

        #self.policy=nn.Sequential(nn.Linear(4,4),
        #                          nn.Tanh(),
        #                          nn.Linear(4,2),
        #                          nn.Tanh())
        self.max_recent_history=20

        self.gyros=deque([],maxlen=self.max_recent_history)
        self.accels=deque([],maxlen=self.max_recent_history)
        self.headings=deque([],maxlen=self.max_recent_history)
        #self.last_gyro=np.zeros(3) #last gyroscope reading
        #self.last_accel=np.zeros(3) #last accelerometer reading
        self.next_motor_end_time=0
        
        
    def get_keys(self):
        return []
    
    def get_name(self):
        return "VisualMotionCalibGyrus"
    
    def holding_still(self,timestamp): #return true if I think I'm holding still
        if len(self.gyros)==0:
            return False
        gyros,accels,heading=self.get_closest_kinematics(timestamp)
        if np.linalg.norm(gyros)>0.03:
            return False
        if np.linalg.norm(accels)>0.15:
            return False
        return True
    

    
    def get_closest_value(self,timestamp,mylist):
        #for mylist ordered by [time,value], return the value that is closest to the input timestamp
        if len(mylist)==0:
            return None
        if timestamp<=mylist[0][0]:
            return mylist[0][1]
        
        first_val=mylist[0]
        second_val=None
        for val in mylist:
            if timestamp>val[0]:
                second_val=val
            else:
                first_val=val
        if abs(first_val[0]-timestamp)<abs(second_val[0]-timestamp):
            return first_val[1]
        else:
            return second_val[1]
    
    def get_closest_kinematics(self,timestamp):
        return self.get_closest_value(timestamp,self.gyros),self.get_closest_value(timestamp,self.accels),self.get_closest_value(timestamp,self.headings)

        
    def read_message(self,message):
        if 'packets' in message: #rotation etc
            for packet in message['packets']:
                self.gyros.append( [packet["gyroscope_timestamp"],packet["gyroscope"]])
                self.accels.append( [packet["acceleration_timestamp"],packet["acceleration"]])
                euler_angles=toeuler(packet["rotation_vector"])
                self.headings.append( [packet["rotation_vector_timestamp"],[euler_angles[2]]])
            
        if 'detections' in message:
            timestamp=message["image_timestamp"]
            dets=get_unique_tagged_objects(message["detections"],[32,11])
            if not self.holding_still(timestamp): #only log when holding still
                return
            print("ok")
            




In [5]:

gyrus=VisualMotionCalibGyrus(None)
for message in sensor_log:
    gyrus.read_message(message)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
o